In [29]:
import math
import random
import time

def main():
    # closest_pair_dist의 n log n 개선.
    print('# closest_pair_dist의 n log n 개선.')

    # 예제 입력.
    p = [(2, 3), (12, 30), (40, 50), (5, 1), (12, 10), (3, 4)]

    # x좌표와 y좌표에 대해 정렬한 리스트 생성.
    px = p[:]
    px.sort(key = lambda e: e[0])
    py = p[:]
    py.sort(key = lambda e: e[1])

    brute = closest_pair(p)
    nlogn = closest_pair_dist(px, py, len(p))

    print(f'브루트 포싱:\t{brute}')
    print(f'개선된 분할정복:\t{nlogn}')
    print()

    n = 100
    m = 100

    brute_time_avg = 0
    nlogn_time_avg = 0

    for _ in range(n):
        p = []
        for _ in range(m):
            # 임의의 점 생성.
            p.append((random.random() * 100, random.random() * 100))

        px = p[:]
        px.sort(key = lambda e: e[0])
        py = p[:]
        py.sort(key = lambda e: e[1])

        brute_start = time.time()
        brute = closest_pair(p)
        brute_end = time.time()

        nlogn_start = time.time()
        nlogn = closest_pair_dist(px, py, len(p))
        nlogn_end = time.time()

        brute_time_avg += brute_end - brute_start
        nlogn_time_avg += nlogn_end - nlogn_start

        # print(brute)
        # print(nlogn)
        # print()

        if brute != nlogn:
            print('abort')
            break
    
    brute_time_avg /= n
    nlogn_time_avg /= n
    
    print(f'서로 다른 {m}개의 점을 추출하여 {n}번 수행 완료.')
    print(f'브루트 포싱 평균 수행 시간:\t{brute_time_avg}')
    print(f'개선된 분할정복 평균 수행 시간:\t{nlogn_time_avg}')

In [30]:
# 두 점 사이의 거리 반환.
def dist(p1, p2):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]

    return math.sqrt(dx * dx + dy * dy)

In [31]:
# x에 대한 점들의 리스트와 y에 대한 점들의 리스트를 동시에 고려하여 점 사이 거리의 최솟값 계산.
def closest_pair_dist(points_x, points_y, n):
    # 점의 개수가 3개 이하이면 브루트 포싱을 이용해 최솟값 계산.
    if n <= 3:
        return closest_pair(points_x)

    # x에 대해 정렬된 리스트를 이용하여 x좌표의 중앙값과 그 점에서의 y좌표 추출.
    mid = n // 2
    mid_x = points_x[mid][0]
    mid_y = points_x[mid][1]

    # x좌표를 기준으로 왼쪽과 오른쪽 부분 리스트를 생성한 것처럼 y좌표에 대해서 왼쪽과 오른쪽(또는 아래쪽과 위쪽) 부분 리스트 생성.
    points_y_l = []
    points_y_r = []
    for i in range(len(points_x)):
        point = points_y[i]

        # x좌표 기준 왼쪽 리스트의 점과 기준 y좌표보다 아래쪽에 있는 점 추출.
        if point[0] < mid_x or (point[0] == mid_x and point[1] < mid_y):
            points_y_l.append(point)

        # x좌표 기준 오른쪽 리스트의 점과 기준 y좌표보다 위쪽에 있는 점 추출.
        else:
            points_y_r.append(point)

    dl = closest_pair_dist(points_x[:mid], points_y_l, mid)         # 왼쪽 부분 리스트에 대해서 분할정복 수행.
    dr = closest_pair_dist(points_x[mid:], points_y_r, n - mid)     # 오른쪽 부분 리스트에 대해서 분할정복 수행.
    d = min(dl, dr)     # dl과 dr 중 최솟값을 저장.

    # 중간 띠 구역에 대해서 최솟값 추출.
    pointsm = []
    for i in range(n):
        # 점 사이의 거리가 최소가 될 수 있는 후보를 y좌표를 기준으로 추출.
        if abs(points_y[i][0] - mid_x) < d:
            pointsm.append(points_y[i])
    ds = strip_closest(pointsm, d)      # 띠 구역에 대해서 최솟값 추출.

    return min(d, ds)       # 거리의 최솟값 추출 및 반환.

In [32]:
# 왼쪽 부분 리스트과 오른쪽 부분 리스트의 중간 부분의 점들의 거리 계산.
def strip_closest(points, d):
    n = len(points)
    d_min = d       # 함수를 호출한 함수에서 계산한 최소 거리를 사용.

    for i in range(n):
        j = i + 1

        # y 좌표를 기준으로 생각한 거리가 d_min보다 작으면 수행. (d_min의 후보)
        # y 좌표가 오름차순 정렬되어 있으므로 모든 점에 대해 if 문을 수행하지 않고 while 문으로 대체 가능.
        while j < n and points[j][1] - points[i][1] < d_min:
            dij = dist(points[i], points[j])    # 실제 거리가 d_min보다 작으면 d_min 갱신.
            if dij < d_min:
                d_min = dij
            j += 1

    return d_min

In [33]:
# 점 사이의 거리의 최솟값을 브루트 포싱으로 계산.
def closest_pair(points):
    min_d = float('inf')

    # 모든 가능한 점들의 조합에 대해 거리 계산 및 최솟값 갱신.
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            p1 = points[i]
            p2 = points[j]

            if p1 == p2:
                continue
            d = dist(p1, p2)
            if d < min_d:
                min_d = d

    return min_d

#### **# closest_pair_dist의 시간 복잡도 계산** ####

##### # 사전 작업
closest_pair_dist는 알고리즘 실행 전에 사전 작업을 수행한다.  
입력으로 들어오는 점들의 리스트를 x좌표와 y좌표를 기준으로 정렬하는 작업이 필요하다.  
다음의 코드를 살펴보자.  

> ``` python
> px = p[:]
> px.sort(key = lambda e: e[1])
> py = p[:]
> py.sort(key = lambda e: e[1])
> ```

위의 코드는 입력으로 들어오는 좌표들을 x좌표와 y좌표에 대해 오름차순으로 정렬한다.  
정렬을 위해 파이썬에서 제공하는 정렬 함수를 사용했다. 파이썬에서 제공하는 정렬 함수는 Tim Sort를 이용하는데, Tim Sort의 평균 시간 복잡도는 다음과 같다.
$$O(n\ \log_2 n)_{\ ...\ (1)}$$
Tim Sort의 시간 복잡도의 자세한 분석은 다루지 않는다.  

##### # closest_pair_dist에서 사용하는 알고리즘의 시간 복잡도 분석
아래에서는 closest_pair_dist에서 사용하는 알고리즘에 대한 시간 복잡도를 살펴본다.  
함수의 형태로 사용된 알고리즘과 closest_pair_dist에서 구현한 알고리즘을 모두 살펴본다.  

##### ## 베이스 케이스
closest_pair_dist의 형태가 재귀적이므로 베이스 케이스를 먼저 살표보자.  

> ``` python
> if n <= 3:
>     return closest_pair(points_x)
> ```

베이스 케이스 상황에서 closest_pair 함수를 사용한다.  
closest_pair 함수의 일부는 아래와 같다.  

> ``` python
> for i in range(len(points)):
>     for j in range(i + 1, len(points)):
>         p1 = points[i]
>         p2 = points[j]
> ```

위와 같이 모든 점들에 대해서 이중 반복문을 수행하는 것을 볼 수 있다.  
따라서 점들의 개수를 n이라 가정하면 위의 알고리즘의 시간 복잡도는 다음과 같다.  
$$O(n^2)$$

그런데 위의 알고리즘이 수행되는 시점은 n이 3 이하인 베이스 케이스가 발생한 시점이므로 n을 3이하의 자연수로 고정할 수 있다.  
따라서 구하고자 하는 시간 복잡도는 상수 시간이므로 다음과 같다.  
$$O(1)_{\ ...\ (2)}$$

##### ## 중앙값 추출하기
다음으로 살펴볼 구문은 정렬된 리스트에서 중앙값을 추출하는 과정이다.  
해당 구문은 다음과 같다.  

> ``` python
> mid = n // 2
> mid_x = points_x[mid][0]
> mid_y = points_x[mid][1]
> ```

위의 구문은 상수 시간 내에 처리할 수 있다는 것이 자명하다.  
따라서 구하고자 하는 시간 복잡도는 다음과 같다.  
$$O(1)_{\ ...\ (3)}$$

##### ## y좌표를 기준으로 한 부분 리스트 구하기
정렬된 x좌표에 대한 리스트에서 중앙값을 추출하여 리스트를 두 개로 분할한 것처럼 y좌표에 대해서도 비슷한 작업을 수행한다.  
y좌표에 대해서 왼쪽과 오른쪽(또는 아래쪽과 위쪽) 부분 리스트를 생성하는 코드는 다음과 같다.  

> ``` python
>     points_y_l = []
>     points_y_r = []
>     for i in range(len(points_x)):
>         point = points_y[i]
> ```

빈 리스트를 선언하여 리스트의 끝에 원소를 삽입하는 것을 볼 수 있다.  
이때 반복의 횟수가 len(points\_x)로 설정되어 있는데, 이를 점들의 개수인 n으로 가정하면 구하고자 하는 시간 복잡도는 다음과 같다.  
$$O(n)_{\ ...\ (4)}$$

##### ## 분할정복 수행 - 1
정렬된 리스트를 기반으로 분할정복 기법을 적용하여 문제의 크기를 줄인다.  

> ``` python
> dl = closest_pair_dist(points_x[:mid], points_y_l, mid)
> dr = closest_pair_dist(points_x[mid:], points_y_r, n - mid)
> d = min(dl, dr)
> ```

위의 코드는 분할정복 기법을 적용하여 문제의 크기를 축소, 알고리즘에 효율을 높이는 구문이다.  
분할정복의 시간 복잡도를 바로 계산하기에는 무리가 있으므로 먼저 나머지 알고리즘을 확인한 후 마스터 정리를 이용하여 시간 복잡도를 구한다.  

한편 세 번째 줄에 있는 min은 두 수 중 최솟값을 반환하며 이는 상수 시간에 처리할 수 있다. 
따라서 min의 시간 복잡도는 다음과 같다.  
$$O(1)_{\ ...\ (5)}$$

##### ## y좌표를 기준으로 최솟값이 될 후보군 추려내기.
정렬된 x좌표를 바탕으로 띠 구간을 추려내어 x좌표에 대해서 점의 후보를 추려낸 것처럼 y좌표를 기준으로 최솟값이 되는 후보군을 추출한다.  
이때에는 x좌표의 중앙값을 기준으로 y좌표를 기준으로 정렬한 리스트를 사용한다.  

> ``` python
> pointsm = []
> for i in range(n):
>     if abs(points_y[i][0] - mid_x) < d:
>         pointsm.append(points_y[i])
> ```

먼저 빈 배열을 생성하여 이후에 조건에 부합하는 점을 삽입한다.  
이때 반복 횟수로 설정된 n은 입력으로 들어온 점들의 개수로 이를 n으로 가정하면 위의 구문의 시간 복잡도는 다음과 같다.  
$$O(n)_{\ ...\ (6)}$$

##### ## 띠 구간의 거리 최솟값 구하기
앞선 구문에서 띠 구간에 들어갈 수 있는 후보군을 추려내었다. 다음으로는 후보군에 있는 점들에 대해 거리의 최솟값을 구한다.  

> ``` python
> ds = strip_closest(pointsm, d)
> ```

<span></span>

> ``` python
> def strip_closest(points, d):
>     n = len(points)
>     d_min = d
> 
>     for i in range(n):
> ```

위의 코드는 띠 구간의 거리의 최솟값을 구하는 함수의 일부이다.  
해당 함수에선 반복문을 이용하여 거리의 최솟값을 계산한다. 이때 반복문을 수행하는 횟수가 len(points)로 점들의 개수만큼 반복한다.  
점들의 개수를 n으로 가정하면 구하고자 하는 시간 복잡도는 다음과 같다.  
$$O(n)_{\ ...\ (7)}$$

##### ## 최종 최솟값 결정 및 반환
위에서 계산한 최솟값들을 바탕으로 최종 반환값을 결정한다.  

> ``` python
> return min(d, ds)
> ```

앞서 살펴보았던 min이 등장한다.  
이때 구하고자 하는 시간 복잡도는 다음과 같다.  
$$O(1)_{\ ...\ (8)}$$

##### ## 분할정복 수행 - 2
위에서 계산한 $(1)$~$(8)$을 이용해 시간 복잡도에 대한 식을 세우면 다음과 같다.  
$$T(n) = 2\ T(n/2) + O(n) + O(n) + O(n) + O(1) + O(1) + O(1) + O(1)$$
$$= 2\ T(n/2) + O(n)$$

한편 베이스 케이스를 $n = 3$인 것으로 가정하면 시간 복잡도는 다음과 같다.  
$$T(3) = O(1)$$

위에서 계산한 시간 복잡도와 마스터 정리를 이용하면 구하고자 하는 시간 복잡도는 다음과 같다.  
단, $T(3) = O(1)$에서 $T(3) = 1$으로 가정한다.

$$T(n) = 2\ T(n/2) + O(n)$$
$$a = 2,\ b = 2,\ c = 1,\ O(f(n)) = O(n) → d = 1$$  
$$2 = 2^1 → T(n) = O(n\ log_2 n)$$

##### # 결론
위에서 구한 사전 작업과 본 알고리즘의 시간 복잡도를 계산하면 다음과 같다.  

$$T(n) = O(n\ log_2 n) + O(n\ log_2 n)$$
$$= O(n\ log_2 n)$$

In [34]:
if __name__ == '__main__':
    main()

# closest_pair_dist의 n log n 개선.
브루트 포싱:	1.4142135623730951
개선된 분할정복:	1.4142135623730951

서로 다른 100개의 점을 추출하여 100번 수행 완료.
브루트 포싱 평균 수행 시간:	0.003899903297424316
개선된 분할정복 평균 수행 시간:	0.0006223082542419434
